# MuPDF, Leptonica, and FFI::Platypus

Experimenting with native bindings and image processing.

In [71]:
use v5.16;
use FFI::Platypus;
use FFI::CheckLib qw( find_lib_or_die );
use Alien::MuPDF;
use Alien::Leptonica;

my $ffi = FFI::Platypus->new; undef


The first library that we want to use is MuPDF. This library is used for fast PDF rendering. The documentation is available [here](http://www.mupdf.com/docs/). Let's look at some example code from the documentation:

In [72]:
# magic incantation to highlight the example C code
my $example_uri = "http://www.mupdf.com/docs/example.c";
my $html = `curl -q '$example_uri' 2>/dev/null | highlight -l -s zenburn --css=/dev/null --inline-css -O html -S c`;
$html =~ s/<body[^>]*>/<body>/; # some cleanup to remove background from <body>
IPerl->html( $html )

<!DOCTYPE html>
 
 
 
 Source file 
 
 
 1 // Rendering a page of a PDF document to a PNG image in less than 100 lines. 
 2 
 3 // Compile a debug build of mupdf, then compile and run this example: 
 4 // 
 5 // gcc -g -o build/debug/example -Iinclude docs/example.c \ 
 6 //	build/debug/libmupdf.a build/debug/libmupdf-js-none.a \ 
 7 //	build/debug/libfreetype.a build/debug/libjbig2dec.a \ 
 8 //	build/debug/libjpeg.a build/debug/libopenjpeg.a \ 
 9 //	build/debug/libz.a -lm 
 10 // 
 11 // build/debug/example /path/to/document.pdf 1 200 25 
 12 
 13 // Include the MuPDF header file. 
 14 #include <mupdf/fitz.h> 
 15 
 16 void 
 17 render ( char * filename , int pagenumber , int zoom , int rotation ) 
 18 { 
 19 // Create a context to hold the exception stack and various caches. 
 20 
 21 fz_context * ctx = fz_new_context ( NULL , NULL , FZ_STORE_UNLIMITED ); 
 22 
 23 // Register document handlers for the default file types we support. 
 24 
 25 fz_register_document_handlers ( ctx ); 
 26 
 27 // Open the PDF, XPS or CBZ document. 
 28 
 29 fz_document * doc = fz_open_document ( ctx , filename ); 
 30 
 31 // Retrieve the number of pages (not used in this example). 
 32 
 33 int pagecount = fz_count_pages ( doc ); 
 34 
 35 // Load the page we want. Page numbering starts from zero. 
 36 
 37 fz_page * page = fz_load_page ( doc , pagenumber - 1 ); 
 38 
 39 // Calculate a transform to use when rendering. This transform 
 40 // contains the scale and rotation. Convert zoom percentage to a 
 41 // scaling factor. Without scaling the resolution is 72 dpi. 
 42 
 43 fz_matrix transform ; 
 44 fz_rotate (& transform , rotation ); 
 45 fz_pre_scale (& transform , zoom / 100.0 f , zoom / 100.0 f ); 
 46 
 47 // Take the page bounds and transform them by the same matrix that 
 48 // we will use to render the page. 
 49 
 50 fz_rect bounds ; 
 51 fz_bound_page ( doc , page , & bounds ); 
 52 fz_transform_rect (& bounds , & transform ); 
 53 
 54 // Create a blank pixmap to hold the result of rendering. The 
 55 // pixmap bounds used here are the same as the transformed page 
 56 // bounds, so it will contain the entire page. The page coordinate 
 57 // space has the origin at the top left corner and the x axis 
 58 // extends to the right and the y axis extends down. 
 59 
 60 fz_irect bbox ; 
 61 fz_round_rect (& bbox , & bounds ); 
 62 fz_pixmap * pix = fz_new_pixmap_with_bbox ( ctx , fz_device_rgb ( ctx ), & bbox ); 
 63 fz_clear_pixmap_with_value ( ctx , pix , 0xff ); 
 64 
 65 // A page consists of a series of objects (text, line art, images, 
 66 // gradients). These objects are passed to a device when the 
 67 // interpreter runs the page. There are several devices, used for 
 68 // different purposes: 
 69 // 
 70 //	draw device -- renders objects to a target pixmap. 
 71 // 
 72 //	text device -- extracts the text in reading order with styling 
 73 //	information. This text can be used to provide text search. 
 74 // 
 75 //	list device -- records the graphic objects in a list that can 
 76 //	be played back through another device. This is useful if you 
 77 //	need to run the same page through multiple devices, without 
 78 //	the overhead of parsing the page each time. 
 79 
 80 // Create a draw device with the pixmap as its target. 
 81 // Run the page with the transform. 
 82 
 83 fz_device * dev = fz_new_draw_device ( ctx , pix ); 
 84 fz_run_page ( doc , page , dev , & transform , NULL ); 
 85 fz_free_device ( dev ); 
 86 
 87 // Save the pixmap to a file. 
 88 
 89 fz_write_png ( ctx , pix , "out.png" , 0 ); 
 90 
 91 // Clean up. 
 92 
 93 fz_drop_pixmap ( ctx , pix ); 
 94 fz_free_page ( doc , page ); 
 95 fz_close_document ( doc ); 
 96 fz_free_context ( ctx ); 
 97 } 
 98 
 99 int main ( int argc , char ** argv ) 
 100 { 
 101 char * filename = argc >= 2 ? argv [ 1 ] : "" ; 
 102 int pagenumber = argc > 2 ? atoi ( argv [ 2 ]) : 1 ; 
 103 int zoom = argc > 3 ? atoi ( argv [ 3 ]) : 100 ; 
 104 int rotation = argc > 4 

In [87]:
$ffi->lib( Alien::MuPDF->dynamic_libs ); undef;
use DDP; &p( Alien::MuPDF->new );
use DDP; p $ffi;

use constant { # from mupdf/fitz/context.h
    FZ_STORE_UNLIMITED => 0,
    FZ_STORE_DEFAULT => 256 << 20,
    FZ_VERSION => "1.5",
};
eval { $ffi->type( 'opaque' => $_ ) for qw(fz_context fz_alloc_context fz_locks_context) };

$ffi->attach( fz_new_context => [ "fz_alloc_context", "fz_locks_context", "unsigned int" ] => "fz_context" );
#$ffi->attach( fz_register_document_handlers => ["fz_context"] => 'void' );
my $ctx = fz_new_context( undef, undef, FZ_STORE_UNLIMITED );
#fz_register_document_handlers( $ctx )


Alien::MuPDF  {
    Parents       Alien::Base, Exporter
    Linear @ISA   Alien::MuPDF, Alien::Base, Exporter
    public methods (2) : cflags, Inline
    private methods (0)
    internals: {}
}
FFI::Platypus  {
    public methods (21) : alignof, attach, attach_cast, bootstrap, cast, closure, custom_type, DESTROY, find_lib, find_symbol, function, ignore_not_found, lang, lib, load_custom_type, new, package, sizeof, type, type_meta, types
    private methods (2) : _type_lookup, _type_map
    internals: {
        handles            {
            /usr/local/lib/libfitz.so   59655792
        },
        ignore_not_found   0,
        lang               "C",
        lib                [
            [0] "/usr/local/lib/libfitz.so",
            [1] "/usr/local/lib/libfitz.so",
            [2] "/usr/local/lib/libfitz.so",
            [3] "/usr/local/lib/libfitz.so",
            [4] "/usr/local/lib/libfitz.so"
        ],
        mangler            sub { ... },
        type_map           {
         

In [24]:
$ffi->lib(find_lib_or_die lib => 'lept'); undef;

In [ ]:
use C::Scan;
use Path::Class;

my $fitz_include_dir = dir( Alien::MuPDF->cflags =~ /-I([^ ]*)/);
my $fitz_filename = $fitz_include_dir->file(qw(mupdf fitz.h));

my $scan = C::Scan->new( filename => $fitz_filename, includeDirs => [$fitz_include_dir] );

my $funcs = $scan->get('fdecls');

use DDP; print p $funcs;